In [ ]:
#cleaning the "case" record numbers was more diffcult because some of the case files were
    #larger than a GB. A string can only hold about 1GB. This meant I had to go into the 
    #terminal and split them into 1GB size files in order to operate as I had with the others,
    #by putting all of the information into an "allLines" string.
    #Because there were not multiple files of the same record type, I had to go
    #about iteration and opening/closing files differently.
    
#This notebook does not strip records after adding in commas, doing so will save storage and 
    #keep SQL queries happier. I had to strip them later, when queries weren't working.

In [1]:
import pandas as pd
import numpy as np
import xlrd
from pathlib import Path
import glob
import re
import csv

In [2]:
pathFile = '/media/herbert/AOC_split_data/Case'
pathList = Path(pathFile).glob('*.txt')

excelPathFile = '/media/herbert/AOC_clean_data/Case/Columns_widths/'
excelPathList = Path(excelPathFile).glob('*.xlsx')

In [3]:
def getExcel(recordNum):
    colPath = ('/media/herbert/AOC_clean_data/Case/Columns_widths/AOC_cols_case' +
              recordNum + '.xlsx')
    return colPath

In [4]:
def cleanList(aList):
    for item in aList:
        if (pd.isnull(item)):
            aList.remove(item)
            
    return aList

In [ ]:
#Creates a list of column widths for each record type

In [5]:
def makeWidthsList(widths):
    newWidths = []
    
    for item in widths:
        item = format(item)
        try:
            newItem = re.search('\([0-9]{1,2}\)', item)
        except:
            newItem = null

        cleanList(widths)
        newItem = newItem.group(0)
        newItem = newItem.replace('(', '')
        newItem = newItem.replace(')', '')
        newWidths.append(newItem)
        
    return newWidths

In [ ]:
#Appends column names to first row

In [6]:
def addColumnNames(aList, file):
    s = '|'

    for item in aList:
        item = format(item)
        try:
            item = item.lower()
        except:
            item = null
            
        cleanList(aList)
    
    names = s.join(aList)
    names+='\n'
    
    file.write(names)

In [ ]:
#Because I had to split the files, I had to create two different methods to create the new file
#This one handles the case that there is only one file of a given record type, so I only
    #do not have to worry about closing the file and rewriting over it on the next iteration
    #of that record type.

In [7]:
def handleSingleFile(currentPath, recordNum):
    data = open(currentPath, 'r', encoding = "ISO-8859-1")

    allLines = data.readlines()

    newFileName = '/media/herbert/AOC_clean_data/Case/Final/AOC_case' + recordNum + '.csv'
    newFileName = format(newFileName)

    newFile = open(newFileName, 'w')

    currentColFile = getExcel(recordNum) 

    cols = pd.read_excel(str(currentColFile))

    colNames = cols['Name'].tolist()

    addColumnNames(colNames, newFile)

    widths = cols['Datatype'].tolist()

    newWidths = makeWidthsList(widths)

    for line in allLines:
        s = '|'
        info_bits = []
        place = 0

        for item in newWidths:
            item = int(item)
            info_bits.append(line[place:place+item])
            place = place+item

        line = s.join(info_bits)
        line += '\n'
        
        newFile.write(line)
        
    data.close()
    newFile.close()

In [ ]:
#This handles multiple files of the same record type, so I do not close the new file until
    #all data for that record type is written in the new file.

In [8]:
def handleMultFile(recordNum, multRecordFiles):
    newFileName = '/media/herbert/AOC_clean_data/Case/Final/AOC_case' + recordNum + '.csv'
    newFileName = format(newFileName)

    newFile = open(newFileName, 'w')
    
    currentColFile = getExcel(recordNum) 

    cols = pd.read_excel(str(currentColFile))

    colNames = cols['Name'].tolist()

    addColumnNames(colNames, newFile)

    widths = cols['Datatype'].tolist()

    newWidths = makeWidthsList(widths)
    
    currentRecordList = []
    for file in multRecordFiles:
        if(file[51:53] == recordNum):
            currentRecordList.append(file)
    
    for file in currentRecordList:
        
        data = open(file, 'r', encoding = "ISO-8859-1")

        allLines = data.readlines()
        
        for line in allLines:
            s = '|'
            info_bits = []
            place = 0

            for item in newWidths:
                item = int(item)
                info_bits.append(line[place:place+item])
                place = place+item

            line = s.join(info_bits)
            line += '\n'
            
            newFile.write(line)
            
        data.close()
    newFile.close()
        

In [9]:
multRecordFiles = []
oneRecordFile = []
for path in pathList:
    currentPath = str(path)
    recordNum = currentPath[51:53]
    intRecordNum = int(recordNum)
    
    #checks if there are multiple files for a record number
    if(currentPath[54] == 'a'):
        multRecordFiles.append(currentPath)
    else:
        oneRecordFile.append(currentPath)

#doneList keeps track of files with multiples for the same record type completed
doneList = []
for file in multRecordFiles:
    recordNum = file[51:53]
    
    if recordNum not in doneList:
        doneList.append(recordNum)
        handleMultFile(recordNum, multRecordFiles)
    
for file in oneRecordFile:
    recordNum = file[51:53]
    handleSingleFile(file, recordNum)


60
22
50
23
48
59
28
57
58
53
55
61
65
46
47
37
29
27
38
21
71
72
73
74
17
24
32
33
69
70
36
03
